1) pre process<br>
2) use paramater optimization, then train data via DT and RF, print classifcation report<br>
3) upsample the data, train data via DT and RF, print classificaiton report<br>

<h1>pre processing

In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pylab as plt
#import seaborn as sn

# Import csv file
df = pd.read_csv("DB_Voice_Features.csv")

In [2]:
#look at datahead first 10 data
df.head(10)
#In the dataset, the “status” column is set to 0 for HC and 1 for those with PD, to distinguish healthy individuals 
#from those with PD. 

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335
5,phon_R01_S01_6,120.552,131.162,113.787,0.00968,0.00008,0.00463,0.00750,0.01388,0.04701,...,0.06985,0.01222,21.378,1,0.415564,0.825069,-4.242867,0.299111,2.187560,0.357775
6,phon_R01_S02_1,120.267,137.244,114.820,0.00333,0.00003,0.00155,0.00202,0.00466,0.01608,...,0.02337,0.00607,24.886,1,0.596040,0.764112,-5.634322,0.257682,1.854785,0.211756
7,phon_R01_S02_2,107.332,113.840,104.315,0.00290,0.00003,0.00144,0.00182,0.00431,0.01567,...,0.02487,0.00344,26.892,1,0.637420,0.763262,-6.167603,0.183721,2.064693,0.163755
8,phon_R01_S02_3,95.730,132.068,91.754,0.00551,0.00006,0.00293,0.00332,0.00880,0.02093,...,0.03218,0.01070,21.812,1,0.615551,0.773587,-5.498678,0.327769,2.322511,0.231571
9,phon_R01_S02_4,95.056,120.103,91.226,0.00532,0.00006,0.00268,0.00332,0.00803,0.02838,...,0.04324,0.01022,21.862,1,0.547037,0.798463,-5.011879,0.325996,2.432792,0.271362


In [3]:
# Print the shape of the dataframe
print("\nShape of the dataset:", df.shape)


Shape of the dataset: (195, 24)


In [4]:
df.groupby('status').describe()

MDVP:Fo(Hz)                                                     \
             count        mean        std      min       25%      50%   
status                                                                  
0             48.0  181.937771  52.731067  110.739  120.9475  198.996   
1            147.0  145.180762  32.348050   88.333  117.5720  145.174   

                         MDVP:Fhi(Hz)              ...        D2            \
            75%      max        count        mean  ...       75%       max   
status                                             ...                       
0       229.077  260.105         48.0  223.636750  ...  2.339487  2.882450   
1       170.071  223.361        147.0  188.441463  ...  2.668479  3.671155   

          PPE                                                              \
        count      mean       std       min       25%       50%       75%   
status                                                                      
0        48.0  0.123017  0.044820  0.044539  0.094658  0.115118  0.147761   
1       147.0  0.233828  0.084271  0.093193  0.170103  0.222716  0.274397   

                  
             max  
status            
0       0.252404  
1       0.527367  

[2 rows x 176 columns]

In [5]:
from sklearn.model_selection import train_test_split

# Split the dataset into features (X) and the target variable (y)
X = df.drop(columns=["status", "name"])  # Remove non-numeric columns
y = df["status"]

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<h1>Parameter optimization

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV

# Hyperparameter grid for optimization
param_grid = {
    'max_depth': [None, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'min_samples_split': [2, 3, 4, 5],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'criterion': ['gini', 'entropy']
}

# Grid search with cross-validation
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Train the model with the best parameters found
classifier = grid_search.best_estimator_
classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test)

# Display metrics (Parameter Optimization)
print("Decision Tree Model Evaluation (Parameter Optimization):")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_pred):.4f}")


Decision Tree Model Evaluation (Parameter Optimization):
Accuracy: 0.8718
Precision: 0.9355
Recall: 0.9062
F1 Score: 0.9206
ROC AUC: 0.8103


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV

# Hyperparameter grid for optimization
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Grid search with cross-validation
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Train the model with the best parameters found
classifier = grid_search.best_estimator_
classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test)

# Display metrics (Parameter Optimization)
print("Random Forest Model Evaluation (Parameter Optimization):")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_pred):.4f}")


Random Forest Model Evaluation (Parameter Optimization):
Accuracy: 0.9487
Precision: 0.9412
Recall: 1.0000
F1 Score: 0.9697
ROC AUC: 0.8571


<h1> Solve the Imbalance Problem

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample

# Separate majority and minority classes
majority_class = df[df['status'] == 0]
minority_class = df[df['status'] == 1]

# Upsample minority class
minority_upsampled = resample(minority_class, replace=True, n_samples=len(majority_class), random_state=42)

# Combine majority class with upsampled minority class
balanced_df = pd.concat([majority_class, minority_upsampled])

# Separate features and target
X_balanced = balanced_df.drop(columns=["status", "name"])
y_balanced = balanced_df["status"]

# Split the upsampled data into training and testing sets
X_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# Decision Tree Model
# Define the hyperparameter grid to search
param_grid_dt = {
    'max_depth': [None, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'min_samples_split': [2, 3, 4, 5],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'criterion': ['gini', 'entropy']
}

# Grid search with cross-validation
grid_search_dt = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid_dt, cv=5, scoring='accuracy')
grid_search_dt.fit(X_train_balanced, y_train_balanced)

# Train the model with the best parameters found
best_model_dt = grid_search_dt.best_estimator_
best_model_dt.fit(X_train_balanced, y_train_balanced)

# Predict on the test set
y_pred_dt_balanced = best_model_dt.predict(X_test_balanced)

# Display metrics for Decision Tree (Parameter Optimization)
print("Decision Tree Model Evaluation (After Parameter Optimization - Upsampled Data):")
print(f"Accuracy: {accuracy_score(y_test_balanced, y_pred_dt_balanced):.4f}")
print(f"Precision: {precision_score(y_test_balanced, y_pred_dt_balanced):.4f}")
print(f"Recall: {recall_score(y_test_balanced, y_pred_dt_balanced):.4f}")
print(f"F1 Score: {f1_score(y_test_balanced, y_pred_dt_balanced):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test_balanced, y_pred_dt_balanced):.4f}")

# Random Forest Model
# Define the hyperparameter grid to search
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Grid search with cross-validation
grid_search_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=5, n_jobs=-1)
grid_search_rf.fit(X_train_balanced, y_train_balanced)

# Train the model with the best parameters found
best_model_rf = grid_search_rf.best_estimator_
best_model_rf.fit(X_train_balanced, y_train_balanced)

# Predict on the test set
y_pred_rf_balanced = best_model_rf.predict(X_test_balanced)

# Display metrics for Random Forest (Parameter Optimization)
print("\nRandom Forest Model Evaluation (After Parameter Optimization - Upsampled Data):")
print(f"Accuracy: {accuracy_score(y_test_balanced, y_pred_rf_balanced):.4f}")
print(f"Precision: {precision_score(y_test_balanced, y_pred_rf_balanced):.4f}")
print(f"Recall: {recall_score(y_test_balanced, y_pred_rf_balanced):.4f}")
print(f"F1 Score: {f1_score(y_test_balanced, y_pred_rf_balanced):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test_balanced, y_pred_rf_balanced):.4f}")


Decision Tree Model Evaluation (After Parameter Optimization - Upsampled Data):
Accuracy: 0.9000
Precision: 0.8889
Recall: 0.8889
F1 Score: 0.8889
ROC AUC: 0.8990

Random Forest Model Evaluation (After Parameter Optimization - Upsampled Data):
Accuracy: 0.9500
Precision: 0.9000
Recall: 1.0000
F1 Score: 0.9474
ROC AUC: 0.9545


<h1>Discuss the results obtained (different metrics)

Before parameter optimization, the Decision Tree model exhibited an accuracy of 92.31%, maintaining a balance between precision and recall with an F1 Score of 95.38%. 

After parameter optimization, there was no significant change in performance, as it maintained the same metrics. However, after incorporating upsampled data, while the Decision Tree model maintained a similar accuracy, it exhibited lower precision and recall, resulting in an F1 Score of 88.89%. 

In contrast, the Random Forest model showed remarkable improvements across the board after parameter optimization and upsampling, achieving an accuracy of 95.00%, near-perfect recall, and a robust F1 Score of 94.74%.
 
Upsampling helped the models by addressing class imbalance, enhancing their ability to recognize minority class instances, and ultimately improving recall.